2.
b)
<br>How many samples (dishes) are there in the training set?
<br> Sample number: 39774
<br> How many categories (types of cuisine)?
<br> Category number: 20
<br> How many unique ingredients are there?
<br> Ingredient number: 6714

In [39]:
import json
import numpy as np

with open('train.json', 'r') as f:
     train_raw_data = json.load(f)
with open('test.json', 'r') as f:
     test_raw_data = json.load(f)
cuisines=set([i['cuisine'] for i in data])
ingredients=set()
for i in [i['ingredients'] for i in data]:
    ingredients |= set(i)
print("Sample number:", len(data))
print("Category number:",len(cuisines))
print("Ingredient number:",len(ingredients))

Sample number: 39774
Category number: 20
Ingredient number: 6714


In [36]:
ingredients=list(ingredients)
train_data,train_labels,test_data,test_ids=[],[],[],[]
for sample in train_raw_data:
    sampleArr=[0]*len(ingredients)
    for i in sample['ingredients']:
        sampleArr[ingredients.index(i)]=1
    train_data.append(sampleArr)
    train_labels.append(sample['cuisine'])
for sample in test_raw_data:
    sampleArr=[0]*len(ingredients)
    for i in sample['ingredients']:
        if i in ingredients: sampleArr[ingredients.index(i)]=1
    test_data.append(sampleArr)
    test_ids.append(sample['id'])

e)
##### Gaussian prior 3 fold cross validation:

In [32]:
from sklearn import datasets,cross_validation,linear_model
from sklearn.naive_bayes import GaussianNB,BernoulliNB

for train, test in (cross_validation.KFold(len(train_data), n_folds=3)):
    cv_train_data=np.array(train_data)[train]
    cv_train_labels=np.array(train_labels)[train]
    cv_test_data=np.array(train_data)[test]
    cv_test_labels=np.array(train_labels)[test]
    gnb = GaussianNB().fit(cv_train_data, cv_train_labels)
    y_pred = gnb.predict(cv_test_data)
    print("Number of mislabeled points out of a total %d points : %d" % (cv_test_data.shape[0],(cv_test_labels != y_pred).sum()))

Number of mislabeled points out of a total 13258 points : 8233
Number of mislabeled points out of a total 13258 points : 8181
Number of mislabeled points out of a total 13258 points : 8252


##### Bernoulli prior 3 fold cross validation:

In [33]:
for train, test in (cross_validation.KFold(len(train_data), n_folds=3)):
    cv_train_data=np.array(train_data)[train]
    cv_train_labels=np.array(train_labels)[train]
    cv_test_data=np.array(train_data)[test]
    cv_test_labels=np.array(train_labels)[test]
    bnb = BernoulliNB().fit(cv_train_data, cv_train_labels)
    y_pred = bnb.predict(cv_test_data)
    print("Number of mislabeled points out of a total %d points : %d" % (cv_test_data.shape[0],(cv_test_labels != y_pred).sum()))

Number of mislabeled points out of a total 13258 points : 4187
Number of mislabeled points out of a total 13258 points : 4249
Number of mislabeled points out of a total 13258 points : 4151


In the multivariate Bernoulli event model, features are independent booleans (binary variables) describing inputs.
Bernoulli event model is especially popular for classifying short texts. 
It has the benefit of explicitly modelling the absence of terms. 
However, in this case, the missing ingredients of a cuisine are not considerably important with respect to training the classifier of cuisine. 

In [34]:
for train, test in (cross_validation.KFold(len(train_data), n_folds=3)):
    cv_train_data=np.array(train_data)[train]
    cv_train_labels=np.array(train_labels)[train]
    cv_test_data=np.array(train_data)[test]
    cv_test_labels=np.array(train_labels)[test]
    lr = linear_model.LogisticRegression(multi_class='ovr')
    lr.fit(cv_train_data, cv_train_labels)
    y_pred = lr.predict(cv_test_data)
    print("Number of mislabeled points out of a total %d points : %d" % (cv_test_data.shape[0],(cv_test_labels != y_pred).sum()))

Number of mislabeled points out of a total 13258 points : 2972
Number of mislabeled points out of a total 13258 points : 3021
Number of mislabeled points out of a total 13258 points : 2934


In [38]:
import csv
lr = linear_model.LogisticRegression(multi_class='ovr')
lr.fit(train_data, train_labels)
y_pred = lr.predict(test_data)
with open('submission.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['id','cuisine'])
    for i,label in enumerate(y_pred):
        spamwriter.writerow([test_ids[i],label])